# Mobina Pournemat - 97105833

## Import Libraries

In [56]:
import os
import random 
import tqdm
import nltk, re
from nltk.tokenize import sent_tokenize
from nltk import word_tokenize
import string
from nltk.corpus import stopwords
from nltk.probability import FreqDist
import numpy as np
import itertools
from nltk import FreqDist
import pandas as pd
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag

In [53]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

## Data Loading
This data which extracted from [The 20 Newsgroups data set](http://qwone.com/~jason/20Newsgroups/) is a collection of approximately 20,000 newsgroup documents, partitioned (nearly) evenly across 20 different newsgroups. The 20 newsgroups collection has become a popular data set for experiments in text applications of machine learning techniques, such as text classification and text clustering.  
for this notebook, I chose the space category which contains 593 text files. 

In [13]:
content = []
path = './sci.space'
for file in os.listdir(path):
    filepath = os.path.join(path, file)
    with open(filepath, 'r') as f:
        content.append(f.read())

print(len(content))

593

## Text sample

In [318]:
for x in random.sample(content, 3):
    print(''.join(x))
    print()
    print("="*80)
    print("="*80)
    print()

From: dnadams@nyx.cs.du.edu (Dean Adams)
Subject: Ariane v.56 Mission Data
Organization: University of Denver, Dept. of Math & Comp. Sci.
Lines: 114



ARIANESPACE FLIGHT 56

(Flight V.56 was originally intended to carry the Hughes HS-601 series 
Galaxy IV satellite, but the payload was withdrawn just prior to flight.)

The 56th Ariane launch is now scheduled to place the ASTRA 1C and ARSENE 
satellites into an improved geostationary transfer orbit (GTO), with 
inclination reduced to 5 degrees and apogee altitude increased by 150 km.
This will be the 28th launch of an Ariane 4 and the first in the 42L 
configuration, with 2 liquid strap-on boosters (PAL).  It will be launched
from the newly refurbished Ariane launch complex ELA 2, in Kourou - French 
Guiana.

The launch vehicle performance requirement for this mission is 3,147 kg
of which 2,944 kg represents the satellite mass.  The total vehicle mass
at liftoff is 361,778 kg.


Required Orbit Characteristics:
  Perigee Altitude ..... 

## Data Cleaning & Normalization

As you can see, the data is very dirty so we need to utilize some  normalization methods to make it usable for different tasks.  
But there is a special thing about this dataset which is containing a lot of email addresses. I'm gonna extract those emails and remove them from sentences.

In [17]:
def find_emails(text):
  emails = re.findall(r"[A-Za-z0-9_%+-.]+"
                 r"@[A-Za-z0-9.-]+"
                 r"\.[A-Za-z]{2,5}",text)
  words = text.split()
  sent = []
  for word in words:
    if word in emails:
      continue
    check = 0
    for e in emails:
      if e in word:
        check = 1
        break
    if check == 0:
      sent.append(word)
      
  return ' '.join(sent), emails

In [19]:
def remove_whitespace(text):
    return  " ".join(text.split())

In [20]:
def get_lowercase(text):
  words = word_tokenize(text)
  lowered = [word.lower() for word in words if len(word)>2]
  return ' '.join(lowered)

In [21]:
def remove_punctuation(text):
  result = [char for char in text if char not in string.punctuation]
  return ''.join(result)

In [22]:
def remove_stopwords(text):
  stop_words = stopwords.words('english')
  words = word_tokenize(text)
  filtered = [word for word in words if (word.lower() not in stop_words)]
  return ' '.join(filtered)

In [172]:
sentences = []
for c in content:
  sent = sent_tokenize(c)
  sentences += sent

print(len(sentences))

9366


In [196]:
normalized_sents = []
emails = []
for sent in sentences:
  s = remove_whitespace(sent)
  s, ems = find_emails(s)
  emails += ems
  s = remove_punctuation(s)
  s = remove_stopwords(s)
  s = get_lowercase(s)
  normalized_sents.append(s)

print(len(normalized_sents))

9366


In [25]:
s = sentences[0]
print(s)
s = remove_whitespace(s)
print(s)
s, _ = find_emails(s)
print(s)
s = remove_punctuation(s)
print(s)
s = remove_stopwords(s)
print(s)
s = get_lowercase(s)
print(s)

From: matthew@phantom.gatech.edu (Matthew DeLuca)
Subject: Re: nuclear waste
Organization: The Dorsai Grey Captains
Lines: 15
NNTP-Posting-Host: oit.gatech.edu

In article <844@rins.ryukoku.ac.jp> will@rins.ryukoku.ac.jp (William Reiken) writes:

>	Ok, so how about the creation of oil producing bacteria?
From: matthew@phantom.gatech.edu (Matthew DeLuca) Subject: Re: nuclear waste Organization: The Dorsai Grey Captains Lines: 15 NNTP-Posting-Host: oit.gatech.edu In article <844@rins.ryukoku.ac.jp> will@rins.ryukoku.ac.jp (William Reiken) writes: > Ok, so how about the creation of oil producing bacteria?
From: (Matthew DeLuca) Subject: Re: nuclear waste Organization: The Dorsai Grey Captains Lines: 15 NNTP-Posting-Host: oit.gatech.edu In article (William Reiken) writes: > Ok, so how about the creation of oil producing bacteria?
From Matthew DeLuca Subject Re nuclear waste Organization The Dorsai Grey Captains Lines 15 NNTPPostingHost oitgatechedu In article William Reiken writes  Ok so h

In [27]:
splited_sents = [sent.split() for sent in normalized_sents]

In [29]:
mp_freqdist = FreqDist(itertools.chain(*splited_sents)) 
top20words = mp_freqdist.most_common(30)
print ('%-16s' % 'word', '%-16s' % 'Frequency','%-16s' %  '% of the total')
for topword in top20words:
    percent=(topword[1]/len(splited_sents))*100
    print ('%-16s' % topword[0], '%-16s' % topword[1],'%-16s' %  percent)

word             Frequency        % of the total  
space            1194             12.748238308776424
subject          633              6.758488148622678
organization     613              6.5449498184924195
lines            613              6.5449498184924195
would            553              5.904334828101645
writes           452              4.82596626094384
one              399              4.260089686098654
article          385              4.110612855007474
nasa             382              4.078582105487935
launch           327              3.4913516976297245
like             324              3.459320948110186
also             285              3.042921204356182
nntppostinghost  282              3.0108904548366433
orbit            275              2.936152039291053
first            273              2.914798206278027
moon             271              2.8934443732650013
university       269              2.872090540251975
get              250              2.6692291266282298
data   

As you can see there are lots of uninformative words with high frequencies. Some of these words are appeared in the above list due to the dataset's characteristics; for instance, words like 'subject', 'organization', and 'lines' which are appeared in the headline of most files and don't contain any information about the dataset's specifications. Moreover, the frequency of auxiliary verbs is high too. So I'm going to apply another stop-word removing step.

In [275]:
def second_stopword_removing(text):
  sw = ['subject', 'organization', 'lines']
  aux = ['will', 'would', 'could', 'can', 'may', 'should','might', 'must']
  words = word_tokenize(text)
  filtered = [word for word in words if (word.lower() not in sw+aux)]
  return ' '.join(filtered)

In [31]:
normalized_sents2 = []
for sent in normalized_sents:
  s = second_stopword_removing(sent)
  normalized_sents2.append(s)

print(len(normalized_sents2))

9366


In [32]:
splited_sents2 = [sent.split() for sent in normalized_sents2]

## Dataset's Statistics

In [34]:
sentence_words = list(itertools.chain(*splited_sents2))
print ('%-16s' % 'Number of words', '%-16s' % len(sentence_words))
print ('%-16s' % 'Number of unique words', '%-16s' % len(set(sentence_words)))
avg=np.sum([len(word) for word in sentence_words])/len(sentence_words)
print ('%-16s' % 'Average word length', '%-16s' % avg)
avg_sentence_length_c=np.mean([len(' '.join(sentence)) for sentence in sentence_words])
print ('%-16s' % 'Average sentence length in characters', '%-16s' % avg_sentence_length_c)
print ('%-16s' % 'Longest word', '%-16s' % sentence_words[np.argmax([len(word) for word in sentence_words])])

Number of words  96505           
Number of unique words 15892           
Average word length 6.477021915962903
Average sentence length in characters 11.954043831925807
Longest word     miscforsalewantedmiscwantednewantednywantednjwanted


## Tokenization

In [71]:
tokenized_sents = []
for sent in normalized_sents2:
  words = word_tokenize(sent)
  tokenized_sents.append(words)

print(len(tokenized_sents))

9366


In [72]:
tokenized_sents[:3]

[['matthew',
  'deluca',
  'nuclear',
  'waste',
  'dorsai',
  'grey',
  'captains',
  'nntppostinghost',
  'oitgatechedu',
  'article',
  'william',
  'reiken',
  'writes',
  'creation',
  'oil',
  'producing',
  'bacteria'],
 ['figure', 'make', 'eat', 'make', 'shit'],
 ['comments']]

## Frequency Analysis

In [280]:
all_tokens = list(itertools.chain(*splited_sents2))
dataframe = {}
token_freq = FreqDist(all_tokens).most_common(30)
dataframe['Token'] = [pair[0] for pair in token_freq]
dataframe['Frequency'] = [pair[1] for pair in token_freq]
freq_analysis = pd.DataFrame(dataframe) 
freq_analysis

,Token,Frequency
0,space,1194
1,writes,452
2,one,399
3,article,385
4,nasa,382
5,launch,327
6,like,324
7,also,285
8,nntppostinghost,282
9,orbit,275


## Stemming & Lemmatization

In [282]:
def get_lemmatization(token):
  lemmatizer = WordNetLemmatizer()
  return lemmatizer.lemmatize(token)

In [283]:
def get_stemming(token):
  stemmer  = PorterStemmer()
  return stemmer.stem(token)

In [284]:
lem_tokenized = [[get_lemmatization(token) for token in sent] for sent in tqdm.tqdm(tokenized_sents)]
len(lem_tokenized)

100%|██████████| 9366/9366 [00:00<00:00, 18657.69it/s]


9366

In [285]:
lemStem_tokenized = [[get_stemming(token) for token in sent] for sent in tqdm.tqdm(lem_tokenized)]
len(lemStem_tokenized)

100%|██████████| 9366/9366 [00:03<00:00, 2926.73it/s]


9366

In [286]:
lemStem_tokenized[:3]

[['matthew',
  'deluca',
  'nuclear',
  'wast',
  'dorsai',
  'grey',
  'captain',
  'nntppostinghost',
  'oitgatechedu',
  'articl',
  'william',
  'reiken',
  'write',
  'creation',
  'oil',
  'produc',
  'bacteria'],
 ['figur', 'make', 'eat', 'make', 'shit'],
 ['comment']]

In [287]:
all_tokens = []
for sent in lemStem_tokenized:
  for token in sent:
    all_tokens.append(token)
len(all_tokens)

96505

In [289]:
dataframe = {}
token_freq = FreqDist(all_tokens).most_common(20)
dataframe['Token'] = [pair[0] for pair in token_freq]
dataframe['Frequency'] = [pair[1] for pair in token_freq]
freq_analysis = pd.DataFrame(dataframe) 
freq_analysis

,Token,Frequency
0,space,1195
1,orbit,506
2,write,478
3,launch,470
4,one,413
5,nasa,409
6,use,404
7,articl,402
8,like,360
9,univers,335


## Keyword extraction using TF-IDF

In order to extract keywords, I am going to combine two different approaches. As you can see, most frequent terms are keywords of this context as well. Thus, I can calculate TF-IDF values just for frequent tokens.

In [290]:
frequents = FreqDist(all_tokens).most_common(1000)

In [291]:
terms = [pair[0] for pair in frequents]
tf = [pair[1] for pair in frequents]

In [292]:
doc_sent = []
for c in content:
  doc_sent.append(sent_tokenize(c))

print(len(doc_sent))

593


In [293]:
normalized_doc_sents = []
for doc in doc_sent:
  temp = []
  for sent in doc:
    s = remove_whitespace(sent)
    s, _ = find_emails(s)
    s = remove_punctuation(s)
    s = remove_stopwords(s)
    s = get_lowercase(s)
    s = second_stopword_removing(s)
    temp.append(s)
  normalized_doc_sents.append(temp)

print(len(normalized_doc_sents))

593


In [294]:
tokenized_doc_sents = []
for doc in normalized_doc_sents:
  temp = []
  for sent in doc:
    words = word_tokenize(sent)
    temp.append(words)
  tokenized_doc_sents.append(temp)

print(len(tokenized_doc_sents))

593


In [295]:
lem_tokenized_doc = []

for doc in tokenized_doc_sents:
  sents = []
  for sent in doc:
    tokens = []
    for token in sent:
      tokens.append(get_lemmatization(token))
    sents.append(tokens)
  lem_tokenized_doc.append(sents)

len(lem_tokenized_doc)

593

In [296]:
lemStem_tokenized_doc = []

for doc in lem_tokenized_doc:
  sents = []
  for sent in doc:
    tokens = []
    for token in sent:
      tokens.append(get_stemming(token))
    sents.append(tokens)
  lemStem_tokenized_doc.append(sents)

len(lemStem_tokenized_doc)

593

In [297]:
df = [0 for _ in range(len(terms))]
for i, term in enumerate(terms):
  for doc in lemStem_tokenized_doc:
    for sent in doc:
      if term in sent:
        df[i] += 1
        break
len(df)

1000

In [298]:
dataframe = {}
dataframe['Token'] = terms[:20]
dataframe['TF'] = tf[:20]
dataframe['DF'] = df[:20]
token_tf_df = pd.DataFrame(dataframe)
token_tf_df

,Token,TF,DF
0,space,1195,294
1,orbit,506,160
2,write,478,359
3,launch,470,121
4,one,413,225
5,nasa,409,139
6,use,404,203
7,articl,402,307
8,like,360,217
9,univers,335,219


In [299]:
def get_idf(df, n):
  idf = np.log2(n/df) + 1
  return idf

In [300]:
n = len(lemStem_tokenized_doc)
idf = []
for x in df:
  idf.append(get_idf(x, n))
len(idf)

1000

In [301]:
term_tfidf = {}
for i in range(len(terms)):
  tfidf = tf[i] * idf[i]
  term_tfidf[terms[i]] = tfidf

In [302]:
sorted_pairs = sorted(term_tfidf.items(), key=lambda kv: -kv[1])

## Best 10 Keywords

In [303]:
dataframe = {}
dataframe['Token'] = [pair[0] for pair in sorted_pairs][:10]
dataframe['TF-IDF'] = [pair[1] for pair in sorted_pairs][:10]
token_tf_idf = pd.DataFrame(dataframe)
token_tf_idf

,Token,TF-IDF
0,space,2404.598060
1,launch,1547.721777
2,orbit,1462.319861
3,satellit,1331.437948
4,nasa,1265.015414
5,system,1091.316205
6,mission,1075.081325
7,use,1028.807160
8,one,990.418234
9,moon,963.014510


## Most frequent emails

In [312]:
email_freq = {}
for email in emails:
  if email_freq.get(email) is None:
    email_freq[email] = 0
    continue
  email_freq[email] = email_freq[email] + 1


In [313]:
sorted_emails = sorted(email_freq.items(), key=lambda kv: -kv[1])
dataframe = {}
dataframe['Email'] = [pair[0] for pair in sorted_emails][:10]
dataframe['Frequency'] = [pair[1] for pair in sorted_emails][:10]
email_freq_df = pd.DataFrame(dataframe)
email_freq_df

,Email,Frequency
0,henry@zoo.toronto.edu,91
1,prb@access.digex.com,82
2,baalke@kelvin.jpl.nasa.gov,60
3,nsmca@aurora.alaska.edu,57
4,nsmca@acad3.alaska.edu,32
5,jbh55289@uxa.cso.uiuc.edu,31
6,sysmgr@king.eng.umd.edu,30
7,mccall@mksol.dseg.ti.com,27
8,aws@iti.org,27
9,isu@VACATION.VENARI.CS.CMU.EDU,25
